In [48]:
import pandas as pd

df = pd.read_csv('./Reports/pochta_markers_full.csv',
                 sep=';',
                 encoding='utf-8',
                 header=0 )
unique_markers = df['Маркер'].unique().tolist()

In [52]:
# Forward fill NaN values in 'Маркер' column
df.fillna(method='ffill', inplace=True)

# Create a pivot table with 'Маркер' as columns and 'Маркер - количество совпадений' as values
pivot_df = df.pivot_table(index=['№ п/п', 'ID звонка', 'Оператор', 'Дата звонка', 'Длительность звонка', 'Всего пауз, сек'], columns='Маркер', values='Маркер - количество совпадений').reset_index()

# Reset the index and rename the columns
pivot_df.columns.name = None  # Remove the columns' name

# Fill NaN values with 0 if needed
pivot_df = pivot_df.fillna(0)

# Now, 'pivot_df' contains the desired structure with 'Маркер' values as columns.



In [55]:
pivot_df.to_excel('./dump_markers_count_full.xlsx')

Работаем с получившейся простыней

In [56]:
# Обязательная часть
df = pivot_df
df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
df['Дата'] = pd.to_datetime(df['Дата звонка'], format='%d.%m.%Y %H:%M:%S')
df['Дата'] = df['Дата'].dt.strftime('%d.%m.%Y')
df = df.reset_index(drop=True)

In [58]:
df

,№ п/п,ID звонка,Оператор,Дата звонка,Длительность звонка,"Всего пауз, сек",1. Установка контакта. Приветствие,1.2 Установка контакта. Представление,1.3 Установка контакта. Уточнение имени клиента,1.4 Установка контакта. Уточнение цели клиента,...,💳 Дебетовая карта,💳 ИИС,💳 ИСЖ,💳 НПФ,💳 ПЛ «Мультибонус»,💳 Потребительский кредит + КК,💳 Сберегательный счет,🔹 Завершение диалога,🔹 Начало диалога,Дата
0,1.0,sber_badge_15_016916741440000,Пригода Ольга,10.08.2023 16:29:04,0 days 00:01:00,6.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.08.2023
1,2.0,sber_badge_3_016916741550000,Валгуцкова Наталья,10.08.2023 16:29:15,0 days 00:20:10,750.1,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.08.2023
2,3.0,sber_badge_15_016916757090000,Пригода Ольга,10.08.2023 16:55:09,0 days 00:00:26,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.08.2023
3,4.0,sber_badge_3_016916757240000,Валгуцкова Наталья,10.08.2023 16:55:24,0 days 00:00:09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.08.2023
4,5.0,sber_badge_15_016916757880000,Пригода Ольга,10.08.2023 16:56:28,0 days 00:01:09,30.6,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.08.2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,1198.0,sber_badge_4_016953903390000,Кирия Татьяна,22.09.2023 16:45:39,0 days 00:52:11,1442.6,0.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,22.09.2023
1198,1199.0,sber_badge_2_016953933340000,Варнавина Марина,22.09.2023 17:35:34,0 days 00:43:04,1458.9,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,9.0,22.09.2023
1199,1200.0,sber_badge_18_016953941230000,Тоноян Мелсида,22.09.2023 17:48:43,0 days 01:00:00,2056.1,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,22.09.2023
1200,1201.0,sber_badge_2_016953963960000,Варнавина Марина,22.09.2023 18:26:36,0 days 00:39:32,1045.4,0.0,1.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,5.0,22.09.2023


In [72]:

# Remove "NaN" strings from the list
unique_markers = [x for x in unique_markers if type(x) is str]

In [81]:
rf = df.pivot_table(index='Оператор', values=unique_markers, aggfunc='sum')
rm = df.pivot_table(index='Оператор', values='Длительность звонка', aggfunc='sum')
tm = df.pivot_table(index='Оператор', values='Всего пауз, сек', aggfunc='sum')

In [66]:
rf

,1. Установка контакта. Приветствие,"Всего пауз, сек"
Оператор,,
Бохан Инна,0.0,28.9
Валгуцкова Наталья,15.0,121138.5
Варнавина Марина,11.0,228358.0
Гугнина Валентина,0.0,63105.0
Давтян Мариам,9.0,176642.2
Евтеев Руслан,2.0,265256.7
Еженкова Виктория,0.0,54096.1
Кирия Татьяна,1.0,93986.9
Кичук Павел,4.0,81569.8


In [73]:
unique_markers

['Выявление потребностей',
 'Слова-паразиты',
 '1.4 Установка контакта. Уточнение цели клиента',
 'Работа с возражениям',
 'Вежливость+эмпатия в процессе обслуживания (Стандарты)',
 'Соблюдение стандартов взаимодействия. Вежливость+эмпатия в процессе обслуживания (Стандарты)',
 '4. Работа с возражениями',
 'Соблюдение стандартов взаимодействия. Эмпатия.',
 '4. Работа с возражениями ДК',
 'Эмпатия и Заинтересованность',
 'Нежелательные выражения',
 'Продажа ПБО',
 'Выявление профиля Клиента',
 '1.5 Установка контакта. Определение профиля клиента',
 '6.3 Кросс-продажа. Пенсионная карта',
 'Позитив',
 '7.4 Завершение встречи. Прощание с клиентом ',
 'Прощание',
 'Вежливость',
 '3.1 Презентация основного продукта. Озвучить основные параметры Кредита наличными (КН)',
 '💳 Потребительский кредит + КК',
 'Цель Клиента',
 'Категория КН',
 '⚠️ Нецензурная лексика',
 '3. Презентация основного продукта. Озвучить основные условия и преимущества ДК',
 'Моя карта ',
 'Провоцирование конфликтов',
 'Ко

In [82]:
tm.to_clipboard()